In [53]:
# doccano.json einlesen
import json
import spacy
from spacy import displacy
from spacy.tokens import DocBin


# ----------------------------------
# ------------ Options: ------------
# ----------------------------------

# maximale Anzahl von Wörtern um die die Spangrenze (Ende) nach rechts verschoben werden kann
maxShift = 2
# Soll die spacy-Datei im Corpus des angegebenen Labelschemas gespeichert werden?
saveResults = False
# Bestimmt, ob die Ergebnisse mit displacy angezeigt werden sollen
runDisplacy = False

modelNumber = "4"            # Nummer des Datensatzes, der verwendet werden soll
datasetToTransform = "train" # Name der json-Datei die umgewandelt werden soll


# ----------------------------------
# ------------- Start: -------------
# ----------------------------------

#erstelle Doc mit dem text und füge alle Annotations hinzu 
cvList = []
with open("./corpus/Modell_{}/{}.jsonl".format(modelNumber, datasetToTransform), 'r') as f:
    for text in f:
        cv = json.loads(text)

#Modell importieren
nlp = spacy.blank("de")
db = DocBin()

# geht alles Datenelemente durch und wandelt sie um
for line in cv['lines']:
    text = line['text']
    annotations = line['label']
    doc = nlp(text)
    ents = []
    
    # geht alle Label durch, passt sie wenn nötig an und speichert sie in spaCys Datenformat
    for annotation in annotations:
        span = doc.char_span(annotation[0], annotation[1], label=annotation[2])

        # Brauche die Loop für Fälle die sonst als None gespeichert werden und Errors erzeugen
        # Solche Fälle werden hier abgefangen und die Labelgrenze um bis zu 5 Stellen nach rechts verschoben
        # Wird der Span trotzdem nicht akzeptiert, wird das Label verworfen
        # Beispielfälle: 
          # Data: "... in Release 1905.", gelabelt abr nicht akzeptiert: "Release 1905", akzeptierter Span: "Release 1905." 
          # Data: "QM&EAM", gelabelt abr nicht akzeptiert: "QM" und "EAM", akzeptierter Span: "QM&EAM" 
        canSave = False
        start = int(annotation[0])
        end = int(annotation[1])
        counter = 0
        #verschiebt die Grenze nach hinten, solange der span ungültig ist
        while span == None and counter <= maxShift:
            counter = counter + 1
            end = end + 1
            span = doc.char_span(start, end, label=annotation[2])

        if not span == None:
            canSave = True
        # geht hier rein, falls der span immer noch ungültig ist
        else:
            end = int(annotation[1])
            counter = 0;
            # setzt die Spangrenze zurück und verschiebt sie diesmal nach vorne, solange der span ungültig ist
            while span == None and counter <= maxShift:
                counter = counter + 1
                start = start - 1
                span = doc.char_span(start, end, label=annotation[2])
            if not span == None:
                canSave = True
        # Speichert Label, falls kein Fehler aufgetreten ist
        if canSave:
            ents.append(span)

    # Speichert die Label in spaCys Format
    doc.ents = ents
    db.add(doc)

    #Anschauliche Darstellung der Labels durch Markierung im Text
    if runDisplacy:
        #print(line)
        colors = {"Skill": "DeepSkyBlue", "Tätigkeit": "chartreuse", "Rolle": "LightCoral", "Branche": "MediumPurple"}
        displacy.render(doc, style="ent", jupyter=True, options={"colors": colors})

# Speichert Daten unter dem Namen aus 'datasetToTransform'
if saveResults:
    db.to_disk("./corpus/Modell_{}/{}.spacy".format(modelNumber, datasetToTransform))
